In [14]:
import pandas as pd
import numpy as np
from datetime import datetime
from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tqdm.notebook import tqdm

In [15]:
custom_date_parser = lambda x: datetime.strptime(x, "%Y-%m-%d")

In [16]:
match_list_df = pd.read_csv('csv_data/match_list.csv',parse_dates=['date'],date_parser=custom_date_parser)
#match_list_df = pd.read_csv('csv_data/match_list.csv')

In [17]:
#match_list_train = match_list_df[match_list_df['train_data']==True]
match_list_train=match_list_df

# Jump to Loop

In [5]:
# def convert_date(x):
#     print(x)
#     return datetime.strptime(x, "%Y-%m-%d")

In [6]:
#match_list_train['date']=match_list_train['date'].apply(convert_date,1)

In [7]:
year='2013'
performance_cutoff_date_start = datetime.strptime(year+'-01-01','%Y-%m-%d')
performance_cutoff_date_end = datetime.strptime(year+'-12-31','%Y-%m-%d')

In [8]:
match_list_train=match_list_train[(match_list_train['date']>=performance_cutoff_date_start)&(match_list_train['date']<=performance_cutoff_date_end)]

In [9]:
match_list_train.columns

Index(['match_id', 'date', 'location', 'first_innings', 'second_innings',
       'winner', 'win_by', 'win_dif', 'toss_winner', 'player_of_match',
       'train_data'],
      dtype='object')

In [10]:
match_list_train['win_by'].unique()

array(['runs', 'wickets'], dtype=object)

In [11]:
country_set=set(match_list_train['first_innings'].unique()).union(set(match_list_train['first_innings'].unique()))

In [12]:
country_list = list(country_set)
len(country_list)

13

In [13]:
#selected_country=country_list[0]

In [14]:
# win_count = match_list_train[match_list_train['winner']==selected_country].shape[0]
# matches_played = match_list_train[(match_list_train['first_innings']==selected_country) | (match_list_train['second_inings']==selected_country)].shape[0]
# total_win_by_runs = match_list_train[(match_list_train['win_by']=='runs')]['win_dif'].sum()
# total_win_by_wickets = match_list_train[(match_list_train['win_by']=='wickets')]['win_dif'].sum()
# win_ratio = win_count/matches_played


# win_count,matches_played,win_ratio,total_win_by_runs,total_win_by_wickets

In [15]:
rank_list = []
for selected_country in country_list:
    win_count = match_list_train[match_list_train['winner']==selected_country].shape[0]
    matches_played = match_list_train[(match_list_train['first_innings']==selected_country) | (match_list_train['second_innings']==selected_country)].shape[0]
    total_win_by_runs = match_list_train[(match_list_train['winner']==selected_country) & (match_list_train['win_by']=='runs')]['win_dif'].sum()
    total_win_by_wickets = match_list_train[(match_list_train['winner']==selected_country) & (match_list_train['win_by']=='wickets')]['win_dif'].sum()
    win_ratio = win_count/matches_played
    total_loss_by_runs = match_list_train[((match_list_train['first_innings']==selected_country) | (match_list_train['second_innings']==selected_country))
                    & (match_list_train['winner']!=selected_country)& (match_list_train['win_by']=='runs')]['win_dif'].sum()
    total_loss_by_wickets = match_list_train[((match_list_train['first_innings']==selected_country) | (match_list_train['second_innings']==selected_country))
                    & (match_list_train['winner']!=selected_country)& (match_list_train['win_by']=='wickets')]['win_dif'].sum()
    
    
    rank_dict = {'country':selected_country,
                 'win_ratio':win_ratio,
                 #'total_win_by_runs':total_win_by_runs,
                 #'total_loss_by_runs':total_loss_by_runs,
                 #'total_win_by_wickets':total_win_by_wickets,
                 #'total_loss_by_wickets':total_loss_by_wickets,
                 'effective_win_by_runs':total_win_by_runs-total_loss_by_runs,
                 'effective_win_by_wickets':total_win_by_wickets-total_loss_by_wickets,
                 'matches_played':matches_played,
                 'win_count':win_count,

                }
    rank_list.append(rank_dict)
    

In [16]:
score_df = pd.DataFrame(rank_list).sort_values('win_ratio',ascending=False)
score_df

,country,win_ratio,effective_win_by_runs,effective_win_by_wickets,matches_played,win_count
7,India,0.655172,-319,70,29,19
10,Bangladesh,0.600000,161,-9,5,3
2,Australia,0.578947,333,-12,19,11
8,South Africa,0.541667,320,-8,24,13
3,Ireland,0.500000,88,-7,4,2
4,England,0.500000,-254,9,20,10
11,Sri Lanka,0.500000,170,7,20,10
12,Pakistan,0.482759,232,-6,29,14
0,West Indies,0.444444,-43,-19,18,8
9,New Zealand,0.428571,29,-10,14,6


In [17]:
#plt.scatter(score_df['matches_played'],range(score_df.shape[0]))

In [18]:
#score_df['matches_played'].mean()

In [19]:
#score_df['matches_played'].std()

In [20]:
#score_df['matches_played'].mean()+score_df['matches_played'].std()

In [21]:
scaler = MinMaxScaler()


In [22]:
score_df['score']=scaler.fit_transform(score_df[['win_ratio','effective_win_by_runs','effective_win_by_wickets','matches_played']]).sum(axis=1)

In [23]:
score_df=score_df.sort_values('score',ascending=False)
score_df

,country,win_ratio,effective_win_by_runs,effective_win_by_wickets,matches_played,win_count,score
7,India,0.655172,-319,70,29,19,3.247113
12,Pakistan,0.482759,232,-6,29,14,2.766281
8,South Africa,0.541667,320,-8,24,13,2.750153
2,Australia,0.578947,333,-12,19,11,2.591938
11,Sri Lanka,0.500000,170,7,20,10,2.533738
4,England,0.500000,-254,9,20,10,2.066602
10,Bangladesh,0.600000,161,-9,5,3,1.940646
9,New Zealand,0.428571,29,-10,14,6,1.848664
0,West Indies,0.444444,-43,-19,18,8,1.836775
3,Ireland,0.500000,88,-7,4,2,1.689153


In [24]:
score_df['rank'] = range(1,score_df.shape[0]+1)

In [25]:
score_df

,country,win_ratio,effective_win_by_runs,effective_win_by_wickets,matches_played,win_count,score,rank
7,India,0.655172,-319,70,29,19,3.247113,1
12,Pakistan,0.482759,232,-6,29,14,2.766281,2
8,South Africa,0.541667,320,-8,24,13,2.750153,3
2,Australia,0.578947,333,-12,19,11,2.591938,4
11,Sri Lanka,0.500000,170,7,20,10,2.533738,5
4,England,0.500000,-254,9,20,10,2.066602,6
10,Bangladesh,0.600000,161,-9,5,3,1.940646,7
9,New Zealand,0.428571,29,-10,14,6,1.848664,8
0,West Indies,0.444444,-43,-19,18,8,1.836775,9
3,Ireland,0.500000,88,-7,4,2,1.689153,10


In [154]:
score_df.to_csv('csv_data/country_rank_'+year+'.csv',index=False)

# Loop

In [19]:
for year in tqdm(['2008','2009','2010','2011','2012','2013','2014','2015','2016','2017','2018','2019']):
    performance_cutoff_date_start = datetime.strptime(year+'-01-01','%Y-%m-%d')
    performance_cutoff_date_end = datetime.strptime(year+'-12-31','%Y-%m-%d')
    match_list_year=match_list_train[(match_list_train['date']>=performance_cutoff_date_start)&(match_list_train['date']<=performance_cutoff_date_end)]
    country_set=set(match_list_year['first_innings'].unique()).union(set(match_list_year['first_innings'].unique()))
    country_list = list(country_set)
    #print(country_list)
    country_rank_list = []
    #print(year,country_rank_list)
    #ab=[]
    for selected_country in tqdm(country_list):
        #print(selected_country)
        win_count = match_list_year[match_list_year['winner']==selected_country].shape[0]
        matches_played = match_list_year[(match_list_year['first_innings']==selected_country) | (match_list_year['second_innings']==selected_country)].shape[0]
        total_win_by_runs = match_list_year[(match_list_year['winner']==selected_country) & (match_list_year['win_by']=='runs')]['win_dif'].sum()
        total_win_by_wickets = match_list_year[(match_list_year['winner']==selected_country) & (match_list_year['win_by']=='wickets')]['win_dif'].sum()
        win_ratio = win_count/matches_played
        total_loss_by_runs = match_list_year[((match_list_year['first_innings']==selected_country) | (match_list_year['second_innings']==selected_country))
                        & (match_list_year['winner']!=selected_country)& (match_list_year['win_by']=='runs')]['win_dif'].sum()
        total_loss_by_wickets = match_list_year[((match_list_year['first_innings']==selected_country) | (match_list_year['second_innings']==selected_country))
                        & (match_list_year['winner']!=selected_country)& (match_list_year['win_by']=='wickets')]['win_dif'].sum()


        rank_dict = {'country':selected_country,
                     'win_ratio':win_ratio,
                     #'total_win_by_runs':total_win_by_runs,
                     #'total_loss_by_runs':total_loss_by_runs,
                     #'total_win_by_wickets':total_win_by_wickets,
                     #'total_loss_by_wickets':total_loss_by_wickets,
                     'effective_win_by_runs':total_win_by_runs-total_loss_by_runs,
                     'effective_win_by_wickets':total_win_by_wickets-total_loss_by_wickets,
                     'matches_played':matches_played,
                     'win_count':win_count,

                    }
        #print('dict',selected_country,rank_dict)
        country_rank_list.append(rank_dict)
        #print('updated_list',country_rank_list)
        #ab.append(rank_dict)
    #print(country_rank_list)
        
    score_df = pd.DataFrame(country_rank_list).sort_values('win_ratio',ascending=False)
    scaler = MinMaxScaler()
    
    score_df['score']=scaler.fit_transform(score_df[['win_ratio','effective_win_by_runs','effective_win_by_wickets','matches_played']]).sum(axis=1)
    
    score_df=score_df.sort_values('score',ascending=False)
    score_df['rank'] = range(1,score_df.shape[0]+1)
    score_df.to_csv('csv_data/country_rank_'+year+'.csv',index=False)
    

In [18]:
match_list_train

,match_id,date,location,first_innings,second_innings,winner,win_by,win_dif,toss_winner,player_of_match,train_data
0,251492,2007-01-06,Auckland,Sri Lanka,New Zealand,Sri Lanka,runs,189,Sri Lanka,ST Jayasuriya,True
1,249232,2007-01-12,Melbourne Cricket Ground,England,Australia,Australia,wickets,8,England,AC Gilchrist,True
2,249228,2007-01-14,Hobart,Australia,New Zealand,Australia,runs,105,Australia,A Symonds,True
3,249236,2007-01-16,Hobart,New Zealand,England,England,wickets,3,New Zealand,A Flintoff,True
4,249233,2007-01-19,Brisbane,England,Australia,Australia,wickets,4,England,MEK Hussey,True
...,...,...,...,...,...,...,...,...,...,...,...
1372,1198240,2020-09-16,Manchester,England,Australia,Australia,wickets,3,England,GJ Maxwell,False
1373,1233461,2020-10-30,Rawalpindi Cricket Stadium,Pakistan,Zimbabwe,Pakistan,runs,26,Pakistan,BRM Taylor,False
1374,1233462,2020-11-01,Rawalpindi Cricket Stadium,Zimbabwe,Pakistan,Pakistan,wickets,6,Zimbabwe,Iftikhar Ahmed,False
1375,1223955,2020-11-27,Sydney Cricket Ground,Australia,India,Australia,runs,66,Australia,SPD Smith,False
